In [1]:
import numpy as np
import torch as T
import torch.nn as nn
import os
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical
import pybullet_envs
import gymnasium as gym
import matplotlib.pyplot as plt
import pandas as pd
import gym_trading_env
import time
from tqdm.auto import tqdm
import dill
import shutil
import glob

ENV_WINDOWS = 20
EVAL_PERIOD = 1
RETURN_DIFFERENCES = []
EVAL_RETURN_DIFFERENCES = []

DEVICE = T.device("cuda:0" if T.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [2]:
class ReplayBuffer:
    def __init__(self, max_size, state_dim, episodes):
        self.max_size = max_size
        self.mem_size = max_size
        self.state_dim = state_dim
        self.episodes = episodes

        self.calibrated = False
        self.mem_ctr = 0
        self.n = 0

        self.states = T.zeros((self.mem_size, state_dim), dtype=T.float32, device=DEVICE)
        self.actions = T.zeros(self.mem_size, dtype=T.int64, device=DEVICE)
        self.next_states = T.zeros((self.mem_size, state_dim), dtype=T.float32, device=DEVICE)
        self.rewards = T.zeros(self.mem_size, dtype=T.float32, device=DEVICE)
        self.dones = T.zeros(self.mem_size, dtype=T.bool, device=DEVICE)

    def append(
        self,
        states: np.ndarray,
        action: int,
        reward: float,
        next_states: np.ndarray,
        done: bool,
    ):
        idx = self.mem_ctr % self.mem_size

        self.states[idx] = T.Tensor(
            np.reshape(states, (np.multiply(*states.shape)))
        ).to(DEVICE)
        self.actions[idx] = action
        self.next_states[idx] = T.Tensor(
            np.reshape(next_states, (np.multiply(*next_states.shape)))
        ).to(DEVICE)
        self.rewards[idx] = reward
        self.dones[idx] = done

        self.mem_ctr += 1

        if self.mem_ctr <= self.mem_size:
            self.n = self.mem_ctr

    def sample(self, batch_size: int):
        weights = T.ones(self.n, device=DEVICE).expand(batch_size, -1)
        batch = T.multinomial(weights, 1, replacement=False).reshape(batch_size)

        states_batch = self.states[batch]
        actions_batch = self.actions[batch]
        rewards_batch = self.rewards[batch]
        next_states_batch = self.next_states[batch]
        dones_batch = self.dones[batch]

        return (
            states_batch,
            actions_batch,
            rewards_batch,
            next_states_batch,
            dones_batch,
        )
    
    def update_size(self):
        if (
            self.episodes != None
            and not self.calibrated
            and self.episodes * self.n < self.max_size
        ):
            self.calibrated = True
            self.mem_size = self.episodes * self.n

            self.states = self.states[:self.mem_size]
            self.actions = self.actions[:self.mem_size]
            self.next_states = self.next_states[:self.mem_size]
            self.rewards = self.rewards[:self.mem_size]
            self.dones = self.dones[:self.mem_size]
            print(f"Resized buffer to {self.mem_size}")

In [3]:
class CriticNetwork(nn.Module):
    def __init__(
        self,
        beta,
        input_dim,
        n_actions,
        fc1_dims=256,
        fc2_dims=256,
        name="critic",
        chkpt_dir="tmp/sac",
        batch_size=64
    ):
        super(CriticNetwork, self).__init__()
        self.input_dim = input_dim
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name + "_sac")
        self.batch_size = batch_size

        self.fc1 = nn.Linear(self.input_dim + n_actions, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.q = nn.Linear(self.fc2_dims, 1)

        self.optimizer = optim.Adam(self.parameters(), lr=beta)

        self.to(DEVICE)

    def forward(self, state: T.Tensor, action: T.Tensor):
        action = action.reshape((self.batch_size, 1))
        action_value = self.fc1(T.cat([state, action], dim=1))
        action_value = F.relu(action_value)
        action_value = self.fc2(action_value)
        action_value = F.relu(action_value)

        q = self.q(action_value)

        return q

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

In [4]:
class ValueNetwork(nn.Module):
    def __init__(
        self,
        beta,
        input_dim,
        fc1_dims=256,
        fc2_dims=256,
        name="value",
        chkpt_dir="tmp/sac",
    ):
        super(ValueNetwork, self).__init__()
        self.input_dim = input_dim
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name + "_sac")

        self.fc1 = nn.Linear(self.input_dim, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, fc2_dims)
        self.v = nn.Linear(self.fc2_dims, 1)

        self.optimizer = optim.Adam(self.parameters(), lr=beta)

        self.to(DEVICE)

    def forward(self, state):
        state_value = self.fc1(state)
        state_value = F.relu(state_value)
        state_value = self.fc2(state_value)
        state_value = F.relu(state_value)

        v = self.v(state_value)

        return v

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

In [5]:
class GumbelSoftmax(RelaxedOneHotCategorical):
    '''
    A differentiable Categorical distribution using reparametrization trick with Gumbel-Softmax
    Explanation http://amid.fish/assets/gumbel.html
    NOTE: use this in place PyTorch's RelaxedOneHotCategorical distribution since its log_prob is not working right (returns positive values)
    Papers:
    [1] The Concrete Distribution: A Continuous Relaxation of Discrete Random Variables (Maddison et al, 2017)
    [2] Categorical Reparametrization with Gumbel-Softmax (Jang et al, 2017)
    '''

    def sample(self, sample_shape=T.Size()):
        '''Gumbel-softmax sampling. Note rsample is inherited from RelaxedOneHotCategorical'''
        u = T.empty(self.logits.size(), device=self.logits.device, dtype=self.logits.dtype).uniform_(0, 1)
        noisy_logits = self.logits - T.log(-T.log(u))
        return T.argmax(noisy_logits, dim=-1)

    def log_prob(self, value):
        '''value is one-hot or relaxed'''
        if value.shape != self.logits.shape:
            value = F.one_hot(value.long(), self.logits.shape[-1]).float()
            assert value.shape == self.logits.shape
        return - T.sum(- value * F.log_softmax(self.logits, -1), -1)

In [6]:
class ActorNetwork(nn.Module):
    def __init__(
        self,
        alpha,
        input_dim,
        max_action,
        fc1_dims=256,
        fc2_dims=256,
        n_actions=2,
        name="actor",
        chkpt_dir="tmp/sac",
    ):
        super(ActorNetwork, self).__init__()
        self.input_dim = input_dim
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name + "_sac")
        self.max_action = max_action
        self.reparam_noise = 1e-6

        self.fc1 = nn.Linear(self.input_dim, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.mu = nn.Linear(self.fc2_dims, self.n_actions)
        self.sigma = nn.Linear(self.fc2_dims, self.n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)

        self.to(DEVICE)

    def forward(self, state):
        prob = self.fc1(state)
        prob = F.relu(prob)
        prob = self.fc2(prob)
        prob = F.relu(prob)

        mu = self.mu(prob)
        sigma = self.sigma(prob)

        sigma = T.clamp(sigma, min=self.reparam_noise, max=1)

        return mu, sigma

    def sample_normal(self, state, reparameterize=True):
        mu, sigma = self.forward(state)
        probabilities = Normal(mu, sigma)

        if reparameterize:
            actions = probabilities.rsample()
        else:
            actions = probabilities.sample()

        action = T.tanh(actions) * T.tensor(self.max_action).to(DEVICE)
        log_probs = probabilities.log_prob(actions)
        log_probs -= T.log(1 - action.pow(2) + self.reparam_noise)
        log_probs = log_probs.sum(1, keepdim=True)

        return action, log_probs

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

In [7]:
class Agent:
    def __init__(
        self,
        positions,
        alpha=0.0003,
        beta=0.0003,
        input_dims=[8],
        gamma=0.99,
        n_actions=2,
        max_size=1000000,
        tau=0.005,
        layer1_size=256,
        layer2_size=256,
        batch_size=256,
        reward_scale=2,
        buffer_episodes=None
    ):
        self.positions = np.array(positions)
        self.gamma = gamma
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_dims, buffer_episodes)
        self.batch_size = batch_size
        self.n_actions = n_actions

        self.actor = ActorNetwork(
            alpha,
            input_dims,
            n_actions=n_actions,
            name="actor",
            max_action=1,
        )
        self.critic_1 = CriticNetwork(
            beta, input_dims, n_actions=n_actions, name="critic_1", batch_size=batch_size
        )
        self.critic_2 = CriticNetwork(
            beta, input_dims, n_actions=n_actions, name="critic_2", batch_size=batch_size
        )
        self.value = ValueNetwork(beta, input_dims, name="value")
        self.target_value = ValueNetwork(beta, input_dims, name="target_value")

        self.scale = reward_scale
        self.update_network_parameters(tau=1)

    def choose_action(self, observation):
        state = T.Tensor(np.array(observation)).to(DEVICE)
        
        actions, _ = self.actor.sample_normal(state.reshape((1, np.multiply(*state.shape))), reparameterize=False)

        # return actions.cpu().detach().numpy()[0]

        action_cont = actions.cpu().detach().numpy()[0]
        action_disc = np.argmin(np.abs(self.positions-action_cont))
        return action_disc

    def remember(self, state, action, reward, new_state, done):
        self.memory.append(state, action, reward, new_state, done)
    
    def update_buffer_size(self):
        self.memory.update_size()

    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        target_value_params = self.target_value.named_parameters()
        value_params = self.value.named_parameters()

        target_value_state_dict = dict(target_value_params)
        value_state_dict = dict(value_params)

        for name in value_state_dict:
            value_state_dict[name] = (
                tau * value_state_dict[name].clone()
                + (1 - tau) * target_value_state_dict[name].clone()
            )

        self.target_value.load_state_dict(value_state_dict)

    def save_models(self):
        print(".... saving models ....")
        self.actor.save_checkpoint()
        self.value.save_checkpoint()
        self.target_value.save_checkpoint()
        self.critic_1.save_checkpoint()
        self.critic_2.save_checkpoint()

    def load_models(self):
        print(".... loading models ....")
        self.actor.load_checkpoint()
        self.value.load_checkpoint()
        self.target_value.load_checkpoint()
        self.critic_1.load_checkpoint()
        self.critic_2.load_checkpoint()

    def learn(self):
        if self.memory.n < self.batch_size:
            return

        state, action, reward, new_state, done = self.memory.sample(self.batch_size)

        value = self.value(state).view(-1)
        value_ = self.target_value(new_state).view(-1)
        value_[done] = 0.0

        actions, log_probs = self.actor.sample_normal(state, reparameterize=False)
        log_probs = log_probs.view(-1)
        q1_new_policy = self.critic_1.forward(state, actions)
        q2_new_policy = self.critic_2.forward(state, actions)
        critic_value = T.min(q1_new_policy, q2_new_policy)
        critic_value = critic_value.view(-1)

        self.value.optimizer.zero_grad()
        value_target = critic_value - log_probs
        value_loss = 0.5 * F.mse_loss(value, value_target)
        value_loss.backward(retain_graph=True)
        self.value.optimizer.step()

        actions, log_probs = self.actor.sample_normal(state, reparameterize=True)
        log_probs = log_probs.view(-1)
        q1_new_policy = self.critic_1.forward(state, actions)
        q2_new_policy = self.critic_2.forward(state, actions)
        critic_value = T.min(q1_new_policy, q2_new_policy)
        critic_value = critic_value.view(-1)

        actor_loss = log_probs - critic_value
        actor_loss = T.mean(actor_loss)
        self.actor.optimizer.zero_grad()
        actor_loss.backward(retain_graph=True)
        self.actor.optimizer.step()

        self.critic_1.optimizer.zero_grad()
        self.critic_2.optimizer.zero_grad()
        q_hat = self.scale * reward + self.gamma * value_
        q1_old_policy = self.critic_1.forward(state, action).view(-1)
        q2_old_policy = self.critic_2.forward(state, action).view(-1)
        critic_1_loss = 0.5 * F.mse_loss(q1_old_policy, q_hat)
        critic_2_loss = 0.5 * F.mse_loss(q2_old_policy, q_hat)

        critic_loss = critic_1_loss + critic_2_loss
        critic_loss.backward()
        self.critic_1.optimizer.step()
        self.critic_2.optimizer.step()

        self.update_network_parameters()


In [8]:
# Utilities
def preprocess(df: pd.DataFrame):
    df["feature_close"] = df["close"].pct_change()
    df["feature_open"] = df["open"] / df["close"]
    df["feature_high"] = df["high"] / df["close"]
    df["feature_low"] = df["low"] / df["close"]
    df["feature_volume"] = df["volume"] / df["volume"].rolling(7 * 24).max()
    df.dropna(inplace=True)
    return df


def reward_function(history):
    log_portfolio = np.log(
        history["portfolio_valuation", -1] / history["portfolio_valuation", -2]
    )
    log_market = np.log(history["data_close", -1] / history["data_close", -2])
    reward = log_portfolio - log_market
    return 1000 * reward


def make_env(data_dir, positions, verbose=True):
    env = gym.make(
        "MultiDatasetTradingEnv",
        dataset_dir=data_dir,
        positions=positions,
        preprocess=preprocess,
        reward_function=reward_function,
        windows=ENV_WINDOWS,
        initial_position=0.0,
        trading_fees=0.18 / 100,  # 0.18% per stock buy / sell (BTCTurk fees)
        portfolio_initial_value=1_000,  # in USD
        verbose=verbose,
    )
    env.unwrapped.add_metric(
        "market_return_raw",
        lambda history: round(
            100 * (history["data_close", -1] / history["data_close", 0] - 1), 2
        ),
    )
    env.unwrapped.add_metric(
        "portfolio_return_raw",
        lambda history: round(
            100
            * (
                history["portfolio_valuation", -1] / history["portfolio_valuation", 0]
                - 1
            ),
            2,
        ),
    )
    env.unwrapped.add_metric(
        "position_changes",
        lambda history: 100
        * np.sum(np.diff(history["position"]) != 0)
        / len(history["position"]),
    )
    # env.unwrapped.add_metric("Episode Length", lambda history: len(history))

    return env


def plot_scores(
    values: list,
    y_axis_label: str,
    label: str,
    file: str,
    title: str,
    values2: list = None,
    label2: str = None,
    hlines: list = [0],
    vlines=[],
):
    x = np.arange(len(values)) + 1
    if values2 != None:
        y_max = max(max(values), max(values2), 0)
        y_min = min(min(values), min(values2), 0)
        x_eval = (np.arange(len(values2)) + 1) * EVAL_PERIOD
    else:
        y_max = max(max(values), 0)
        y_min = min(min(values), 0)
    y_ticks = np.arange(y_min, y_max, (y_max - y_min) / 11)

    plt.figure(figsize=(15, 9))
    plt.plot(x, values, color="C0", label=label)

    if values2 != None:
        plt.plot(x_eval, values2, color="C1", label=label2)

    for y in hlines:
        plt.axhline(y=y, color="black", lw=0.3)
    for x in vlines:
        plt.axvline(x=x, color="black", lw=0.1)

    plt.yticks(y_ticks)
    plt.xlabel("Episodes")
    plt.ylabel(y_axis_label)
    plt.legend()
    plt.title(title)
    plt.tight_layout()
    plt.savefig(file)
    plt.close()

In [9]:
EPISODES = 2000

In [10]:
var_files = glob.glob("./sac_vars/*")
render_files = glob.glob("./sac_render/*")
for file in var_files + render_files:
    os.remove(file)

# positions = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
positions = list(np.arange(0, 1, 0.01))

training_env = make_env("./data/training/*.pkl", positions, False)
eval_env = make_env("./data/testing/*.pkl", positions, False)
agent = Agent(
    positions=positions,
    input_dims=np.multiply(*training_env.observation_space.shape),
    n_actions=1,
    batch_size=256,
    alpha=0.001,
    max_size=int(1e6),
    buffer_episodes=500,
    tau=0.0005,
)
AVG_EPISODES = int(EPISODES * 0.1)

best_eval_score = -np.inf
scores, eval_scores = [], []
avg_scores, eval_avg_scores = [], []
times_taken, times_eval_taken = [], []

In [11]:
for episode in tqdm(range(1, EPISODES+1)):
    time_start = time.perf_counter()
    
    state, info = training_env.reset()
    done, truncated = False, False
    score, length = 0, 0
    while not (done or truncated):
        action = agent.choose_action(state)
        next_state, reward, done, truncated, info = training_env.step(action)
        score += reward
        agent.remember(state, action, reward, next_state, done)
        agent.learn()
        state = next_state
        length += 1
    agent.update_buffer_size()
    scores.append(score)
    avg_scores.append(np.mean(scores[-AVG_EPISODES:]))
    
    metrics = training_env.unwrapped.results_metrics
    market_return = metrics["market_return_raw"]
    portfolio_return = metrics["portfolio_return_raw"]
    position_changes = metrics["position_changes"]
    RETURN_DIFFERENCES.append(portfolio_return - market_return)        
    
    time_taken = time.perf_counter() - time_start
    times_taken.append(time_taken)
    tqdm.write(f"\nEpisode: {episode:>4}, Score: {score:>9.3f}, Average Score: {avg_scores[-1]:>9.3f}, Market Return: {market_return:>7.2f}%, Portfolio Return: {portfolio_return:>8.2f}%, Position Changes: {position_changes:>6.2f}%, Length: {length}, Time Taken: {time_taken:>6.2f}s")
    
    
    # Evaluation
    time_eval_start = time.perf_counter()
    
    eval_state, eval_info = eval_env.reset()
    eval_done, eval_truncated = False, False
    eval_score, eval_length = 0, 0
    while not (eval_done or eval_truncated):
        eval_action = agent.choose_action(eval_state)
        eval_next_state, eval_reward, eval_done, eval_truncated, eval_info = eval_env.step(eval_action)
        eval_score += eval_reward
        eval_state = eval_next_state
        eval_length += 1
    eval_scores.append(eval_score)
    eval_avg_scores.append(np.mean(eval_scores[-AVG_EPISODES:]))
    
    eval_metrics = eval_env.unwrapped.results_metrics
    eval_market_return = eval_metrics["market_return_raw"]
    eval_portfolio_return = eval_metrics["portfolio_return_raw"]
    eval_position_changes = eval_metrics["position_changes"]
    EVAL_RETURN_DIFFERENCES.append(eval_portfolio_return - eval_position_changes)
    
    time_eval_taken = time.perf_counter() - time_eval_start
    times_eval_taken.append(time_eval_taken)
    tqdm.write(f"Evaluating...  Score: {eval_score:>9.3f}, Average Score: {eval_avg_scores[-1]:>9.3f}, Market Return: {eval_market_return:>7.2f}%, Portfolio Return: {eval_portfolio_return:>8.2f}%, Position Changes: {eval_position_changes:>6.2f}%, Length: {eval_length}, Time Taken: {time_eval_taken:>6.2f}s")
    
    eval_env.unwrapped.save_for_render("./sac_render/")
    render_saves = glob.glob("./sac_render/*.pkl")
    latest_render_save = max(render_saves, key=os.path.getctime)
    os.rename(latest_render_save, f"./sac_render/sac_E{episode:04}_[{eval_score:+08.2f}].pkl")
    
    if (len(eval_avg_scores) > AVG_EPISODES) and (eval_avg_scores[-1] > best_eval_score):
        best_eval_score = eval_avg_scores[-1]
        print("Saving Variables...", end="\r")
        dill.dump_module(f"sac_vars/sac_E{episode:04}_[{eval_score:+08.2f}].dill")
        shutil.copy(
            f"sac_vars/sac_E{episode:04}_[{eval_score:+08.2f}].dill",
            "sac_vars/sac_latest.dill",
        )
        shutil.copy(
            f"./sac_render/sac_E{episode:04}_[{eval_score:+08.2f}].pkl",
            f"./sac_render/sac_best_E{episode:04}_[{best_eval_score:+08.2f}].pkl",
        )
    
    time_taken = time.perf_counter() - time_start
    print(f"Total Time Taken: {time_taken:.2f}s")
    
    plot_scores(
        values=scores,
        values2=eval_scores,
        y_axis_label="Score",
        label="Training Score",
        label2="Evaluation Score",
        title="Scores vs Episodes",
        file="SAC_Scores.png",
    )
    plot_scores(
        values=RETURN_DIFFERENCES,
        values2=EVAL_RETURN_DIFFERENCES,
        y_axis_label="Return Difference (%)",
        label="Training Return Difference",
        label2="Evaluation Return Difference",
        title="Return Differences vs Episodes",
        file="SAC_Returns.png",
    )
    plot_scores(
        values=times_taken,
        y_axis_label="Training Time",
        label="Training Time",
        title="Training Times vs Episodes",
        file="SAC_Times.png",
        hlines=[],
    )

  0%|          | 0/2000 [00:00<?, ?it/s]

Resized buffer to 274000

Episode:    1, Score:  -353.884, Average Score:  -353.884, Market Return:   27.24%, Portfolio Return:   -10.68%, Position Changes:  80.51%, Length: 548, Time Taken:   3.74s
Evaluating...  Score:  -563.666, Average Score:  -563.666, Market Return:   18.07%, Portfolio Return:   -32.81%, Position Changes:  88.26%, Length: 544, Time Taken:   0.56s
Total Time Taken: 4.31s

Episode:    2, Score:  -343.902, Average Score:  -348.893, Market Return:   10.24%, Portfolio Return:   -21.84%, Position Changes:  72.64%, Length: 540, Time Taken:   6.34s
Evaluating...  Score:  -306.524, Average Score:  -435.095, Market Return:   18.07%, Portfolio Return:   -13.10%, Position Changes:  33.58%, Length: 544, Time Taken:   0.58s
Total Time Taken: 6.92s

Episode:    3, Score:   -94.511, Average Score:  -264.099, Market Return:   10.67%, Portfolio Return:     0.69%, Position Changes:   4.81%, Length: 540, Time Taken:   6.37s
Evaluating...  Score:  -165.899, Average Score:  -345.363, 